In [1]:
import os
import pandas as pd


In [2]:
import pandas as pd
# 假设文件名为 'your_file.csv'
file_path = '/share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/generalization_description_vectors.csv'
# 读取文件
data = pd.read_csv(file_path)
# 确保 Description 列的内容全为字符串
if 'Description' in data.columns:
    data['Description'] = data['Description'].astype(str)
# 保存修改后的文件
data.to_csv(file_path, index=False)
print("All entries in the 'Description' column have been converted to strings.")


All entries in the 'Description' column have been converted to strings.


In [3]:
import pandas as pd
# 读取文件
data = pd.read_csv(file_path)
data.iloc[:, 0] = data.iloc[:, 0].astype(str)  # 将第一列强制为字符串

# 打印原始列名
print("Original columns:", data.columns.values)
# 修改第一列的列名
data.rename(columns={data.columns[0]: "Word"}, inplace=True)
for i in range(5120):
    data.rename(columns={data.columns[i + 1]: f"dim_{i}"}, inplace=True)
# 打印修改后的列名
print("Updated columns:", data.columns.values)
# 保存修改后的文件
data.to_csv(file_path, index=False)

Original columns: ['Word' 'dim_0' 'dim_1' ... 'dim_5117' 'dim_5118' 'dim_5119']
Updated columns: ['Word' 'dim_0' 'dim_1' ... 'dim_5117' 'dim_5118' 'dim_5119']


In [2]:
def merge_csv_files(input_folder, output_file):
    """
    合并指定文件夹中的多个 CSV 文件，生成一个合并后的文件。
    
    参数:
        input_folder (str): 包含 CSV 文件的文件夹路径
        output_file (str): 合并后保存的文件路径
    """
    # 获取文件夹中所有的 CSV 文件
    csv_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(".csv")]
    
    # 初始化一个空的列表，用于存储所有数据框
    data_frames = []

    # 遍历每个 CSV 文件并加载为 DataFrame
    for csv_file in csv_files:
        print(f"正在加载文件: {csv_file}")
        df = pd.read_csv(csv_file)
        data_frames.append(df)

    # 合并所有 DataFrame
    combined_df = pd.concat(data_frames, ignore_index=True)

    # 保存到输出文件
    combined_df.to_csv(output_file, index=False)
    print(f"所有文件已合并并保存到: {output_file}")


In [3]:
# 示例用法
input_folder = "/share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors"  # 替换为包含 CSV 文件的文件夹路径
output_file = "/share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/merged_vectors.csv"  # 替换为合并后的文件路径
merge_csv_files(input_folder, output_file)

正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/PV_mat.csv
正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/PV_terms.csv
正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/ChatGPT_describe_PVmaterials_vectors.csv
正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/expertise_description_vectors-Copy1.csv
正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors/generalization_description_vectors-Copy1.csv
正在加载文件: /share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_ma

In [4]:
import pandas as pd
import numpy as np

input_folder = "/share/home/pwmat/Huggingface_Model_Downloades/model_test_ChemDFM-13B-v1.0/allwords_bandgap_0.9_1.4_data/common_PV_materials_and_vectors"
# 加载 CSV 文件
input_file = input_folder + "/merged_vectors.csv" # 替换为您的实际文件路径
data = pd.read_csv(input_file)

# 提取向量部分（从第二列开始）
description_vectors = data.iloc[:, 1:].values  # 提取向量部分为 NumPy 数组

# 检查数据维度
print(f"数据维度: {description_vectors.shape}")  # 应为 (num_descriptions, 5120)

# 使用加权平均法提取共同特征
mean_vector = np.mean(description_vectors, axis=0)  # 计算每一列的平均值作为共同特征向量

# 检查结果向量
print("Photovoltaic materials 的语义特征向量 (加权平均):")
print(mean_vector)
print("向量维度:", mean_vector.shape)

# 准备保存的 DataFrame，第一列为 "Photovoltaic materials"，后续列为向量值
output_data = pd.DataFrame(
    data=[["PV Materials"] + mean_vector.tolist()],  # 插入词汇及向量
    columns=["Word"] + [f"dim_{i}" for i in range(len(mean_vector))]
)

# 保存到 CSV 文件
output_file = "principal_vector.csv"
output_data.to_csv(output_file, index=False)
print(f"主成分向量已保存到: {output_file}")

数据维度: (446, 5120)
Photovoltaic materials 的语义特征向量 (加权平均):
[-0.25992844 -0.34394482 -0.08905637 ... -0.07522975  0.75947138
  0.09649982]
向量维度: (5120,)
主成分向量已保存到: principal_vector.csv
